# Prepare the food captions with the required format for stage 2 finetuning of MiniGPT-4
#### Required Format:
Refer to the file filter_cap.json from the download link at
https://github.com/Vision-CAIR/MiniGPT-4/blob/main/dataset/README_2_STAGE.md

## Load json file with captions

In [1]:
import json
f = open("/content/finetune_data.json")
captions = json.load(f)
f.close()

## Process food captions into the required format

In [2]:
!pip install unidecode

In [3]:
import unidecode

In [4]:
caption_lst = []
for sample in captions:
  file_name = sample['filename'].lower()
  caption = sample['caption']
  # Handle image ids with '
  file_name = file_name.replace("'", "_")
  # Handle image ids with non english characters (e.g. latin)
  file_name = unidecode.unidecode(file_name)
  caption = unidecode.unidecode(caption)
  # Format of file name: label/image_name
  # Only keep image_name
  slash_idx = file_name.index("/")
  dot_idx = file_name.index(".jpg")
  image_id = file_name[slash_idx+1:dot_idx]
  # Three corrupted images should be removed
  if image_id in ["abalone_0062","corn_dog_0047","hummus_0033"]:
    continue
  caption_dict = {"image_id": image_id, "caption": caption}
  caption_lst.append(caption_dict)
filter_cap = {"annotations":caption_lst}

In [5]:
len(filter_cap['annotations'])

19877

In [6]:
with open("filter_cap.json", 'w') as json_file:
    json.dump(filter_cap, json_file)

## Get a separte file that excludes challenging labels

In [7]:
label_set = {"Aloo_gobi","Baingan_bharta","Chakli","Sambar","Vindaloo","Bon_bon_chicken",
             "Chinese_chicken_salad","Shanghai_fried_noodles","Taro_dumpling","Wonton_noodles",
             "Katsudon","Soba","Tonkotsu_ramen"}
label_set = {label.lower() for label in label_set}

In [8]:
caption_challenging = []
for sample in filter_cap['annotations']:
  if sample['image_id'][:-5] not in label_set:
    caption_challenging.append(sample)
filter_cap_challenging = {"annotations":caption_challenging}

In [9]:
len(filter_cap_challenging['annotations'])

19357

In [10]:
with open("filter_cap_challenging.json", 'w') as json_file_2:
    json.dump(filter_cap_challenging, json_file_2)